In [1]:
import ipyparallel as ipp
from sklearn.externals import joblib

In [144]:
rc = ipp.Client()
dview = rc[:]

dview.execute('import numpy as np')
dview.execute('import time')
dview.execute('from sklearn.externals import joblib')
dview.execute('from scipy.spatial.distance import cdist')
dview.execute('import alfabeto_data.dissertation_images as di')

<AsyncResult: execute>

In [145]:
all_silhouette_ = {}
thenumber_ = 0
for corpus_ in ['alfabeto_continuo', 'bach', 'palestrina']:
    for mode_ in ['major', 'minor']:
        for mc_ in range(5):
            all_silhouette_[thenumber_] = {'corpus': corpus_, 'mode': mode_, 'mc': mc_}
            thenumber_ += 1

In [146]:
@dview.parallel(block=True)
def silhouette_hmm(silhouette_number):
    all_silhouette = {}
    thenumber = 0
    for corpus in ['alfabeto_continuo', 'bach', 'palestrina']:
        for mode in ['major', 'minor']:
            for mcc in range(5):
                all_silhouette[thenumber] = {'corpus': corpus, 'mode': mode, 'mc': mcc}
                thenumber += 1
    corpus_name = all_silhouette[silhouette_number]['corpus']
    mode_name = all_silhouette[silhouette_number]['mode']
    mc = all_silhouette[silhouette_number]['mc']
    K_range = 15
    flipped_data = {}
    for K in range(2, K_range+1):
        all_data = []
        all_labels = []
        l_number = joblib.load('/home/daniel/Desktop/hmm_mc/%s/%s/%s/%s/0.pkl' % (corpus_name, mode_name, K, mc))['test'][0]
        l_numeral = joblib.load('/home/daniel/Desktop/hmm_mc/%s/%s/%s/%s/0.pkl' % (corpus_name, mode_name, K, mc))['train'][2]
        for l in l_number:
            if l[0] < len(l_numeral):
                all_labels.append(l_numeral[l[0]])
            else:
                all_labels.append('o')
        #             print('one didn\'t fit...')
        for x in range(500):
            temp_data = []
            print(x)
            c = joblib.load('/home/daniel/Desktop/hmm_mc/%s/%s/%s/%s/%s.pkl' % (corpus_name, mode_name, K, mc, x))['decode']
            print('decoded')
            for xx in c[1]:
                temp_data.append(xx)
            all_data.append(temp_data)
        numpied = np.swapaxes(np.array(all_data), 0, 1)
    #     print(numpied.shape)
        st = time.time()
#         DIST = cdist(numpied, numpied, 'hamming')
        DIST = (cdist(numpied, numpied, 'hamming')*len(l_number))**2 #Quinn/White
        st2 = time.time()
        flipped_data[K] = di.k_means_simple(DIST, K, all_labels)
    #     print_labels = ['$'+x+'$' for x in all_labels]
    #     di.k_means_data(DIST, K, print_labels, '/home/daniel/Desktop/hmmkmeans/%s_%s.pdf' % (corpus, K))
    #     print('kmeans took', time.time()-st2)
        print(K, '-->', flipped_data[K])
    return {(corpus_name, mode_name, mc): flipped_data}

In [147]:
j = silhouette_hmm.map([mc for mc in range(len(all_silhouette_))])
joblib.dump(j, '/home/daniel/Desktop/silhouette_data_QW.pkl', compress=True)

['/home/daniel/Desktop/silhouette_data_QW.pkl']

In [16]:
all_data = joblib.load('/home/daniel/Desktop/silhouette_data_dict.pkl')

In [10]:
all_data = {}
for x in j:
    for labels, data in x.items():
        if labels[0] not in all_data.keys():
            all_data[labels[0]] = {}
        if labels[1] not in all_data[labels[0]]:
            all_data[labels[0]][labels[1]] = {}
        for K, score in data.items():
            if K not in all_data[labels[0]][labels[1]]:
                all_data[labels[0]][labels[1]][K] = []
            all_data[labels[0]][labels[1]][K].append(score['silhouette'])
        
# print(all_data)
# joblib.dump(all_data, '/home/daniel/Desktop/silhouette_data_dict_QW.pkl', compress=9)
            
# {'alf': {'major': {2: [[]], 3:[[]]}}, {'minor': {2: [[]], 3: [[]]}}, 'palestrina'}

NameError: name 'j' is not defined

In [5]:
import matplotlib.pyplot as plt
import numpy as np

In [17]:
name_dict = {'alfabeto_continuo': 'Alfabeto', 'bach': 'Bach', 'palestrina': 'Palestrina'}
fig = plt.figure()
ax = fig.gca()
for corpus, a in all_data.items():
    for mode, b in a.items():
        if mode == 'major':
            plt.plot([K for K in b.keys()], [np.mean(c) for c in b.values()], label=name_dict[corpus]+' '+mode)
# for corpus, a in all_data.items():
#     for mode, b in a.items():
#         plt.plot([K for K in b.keys()], [np.std(c) for c in b.values()], label=corpus+' '+mode)
#         for K, data in b.items():
#             plt.plot()
plt.legend(loc=1)


ax.set_xticks(range(2, 16))
# ax.set_yticks([0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1])
ax.grid(True)
plt.xlabel('Number of Functions')
plt.ylabel('Silhouette Score')
plt.savefig('/home/daniel/Dropbox/Dissertation/Latex/figures/function/functions_major.pdf', bboxinches='tight')
plt.show()

In [99]:
alf_data_maj = per_number(alf_maj)
alf_data_min = per_number(alf_min)
bach_data_maj = per_number(bach_maj)
bach_data_min = per_number(bach_min)
pal_data_maj = per_number(pal_maj)
pal_data_min = per_number(pal_min)

def per_number(variable):
    jj = []
    for num in range(2, 16):
        num_list = []
        for x in variable:
            for a in x.values():
#                 print(a)
                for i, p in a.items():
#                     print(i, num)
                    if i == num:
#                         print(i, p['silhouette'])
                        num_list.append(p['silhouette'])
            if len(num_list) > 0:
                jj.append(num_list)
#                 jj.append(all_i)
    return jj
                

In [103]:
plt.plot([np.mean(x) for x in alf_data_maj])

In [104]:
plt.show()

In [108]:
alf_maj

[{('alfabeto_continuo',
   'major',
   0): {2: {'completeness': 0.83544563202798272,
    'silhouette': 0.96949812934582758}, 3: {'completeness': 0.83600349226648563,
    'silhouette': 0.98173518931147485}, 4: {'completeness': 0.84891316640904324,
    'silhouette': 0.9054803449345209}, 5: {'completeness': 0.7468733035325118,
    'silhouette': 0.71086716309217024}, 6: {'completeness': 0.83521020007495561,
    'silhouette': 0.64848350668140686}, 7: {'completeness': 0.7059715894053098,
    'silhouette': 0.60426052216769377}, 8: {'completeness': 0.86618678687831752,
    'silhouette': 0.66081271079640858}, 9: {'completeness': 0.90853037032212647,
    'silhouette': 0.73493594782065774}, 10: {'completeness': 0.92371516950301358,
    'silhouette': 0.78068628062951984}, 11: {'completeness': 0.91393387522635927,
    'silhouette': 0.77755966098620555}, 12: {'completeness': 0.87476768298213248,
    'silhouette': 0.77269527397872606}, 13: {'completeness': 0.7218262968312541,
    'silhouette': 0.6348

In [63]:
np.mean(jj)

0.64882730752797235